# Evaluate Normal Model

In [ ]:
from pathlib import Path
from string import ascii_uppercase

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
TEST_DIR = 'DATASET_A_AP/test'

# args
entire_model_s_path = 'saved_model/normal_model' 
model_prefix = 'normal-model'


chart_s_dir = 'charts'

fc_chart_name = f"{model_prefix}-alphabet-failed-predict-count-plotly.jpg"
fc_chart_s_path = f"{chart_s_dir}/{fc_chart_name}"

fc_chart_mpl_name = f"{model_prefix}-alphabet-failed-predict-count-matplotlib.jpg"
fc_chart_mpl_s_path = f"{chart_s_dir}/{fc_chart_mpl_name}"

acc_chart_name = f"{model_prefix}-alphabet-acc-plotly.jpg"
acc_chart_s_path = f"{chart_s_dir}/{acc_chart_name}"

acc_chart_mpl_name = f"{model_prefix}-alphabet-acc-matplotlib.jpg"
acc_chart_mpl_s_path = f"{chart_s_dir}/{acc_chart_mpl_name}"

cm_hs_name = f"{model_prefix}-confusion-matrix-heat-map.jpg"
cm_hm_s_path = f"{chart_s_dir}/{cm_hs_name}"

In [ ]:
# Convert folder to dataframe of images' paths & labels
def get_paths_labels(path, allowed_extension="jpg"):
        Path
        images_dir = Path(path)
        
        filepaths = pd.Series((images_dir.glob(fr'**/*.{allowed_extension}'))).astype(str)
        filepaths.name = "path"
        
        labels = filepaths.str.split("/")[:].str[-2]
        labels.name = "label"

        # Concatenate filepaths and labels
        df = pd.concat([filepaths, labels], axis=1)

        # Shuffle the DataFrame and reset index
        df = df.sample(frac=1).reset_index(drop = True)
        return df

In [ ]:
test_df = get_paths_labels(TEST_DIR)

In [ ]:
test_generator = ImageDataGenerator(rescale=1. / 255.)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='path',
    y_col='label',
    target_size=(28, 28),
    color_mode='grayscale',
    class_mode='categorical',
    # batch_size=1,
    shuffle=False,
)

fc_chart_name = f"{model_prefix}-alphabet-failed-predict-count-plotly.jpg"
fc_chart_s_path = f"{chart_s_dir}/{fc_chart_name}"# reload the entire model

In [ ]:
model = tf.keras.models.load_model(entire_model_s_path)

In [ ]:
model.summary()

### Review Accuracy and Loss

In [ ]:
loss, acc = model.evaluate(test_images)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

### View each class prediction

In [ ]:
alphabet_ls = list(ascii_uppercase)

In [ ]:
pred_alphabet_idxs = np.argmax(model.predict(test_images), -1)

In [ ]:
pred_res = [alphabet_ls[i] for i in pred_alphabet_idxs]

In [ ]:
ground_truths = [alphabet_ls[i] for i in test_images.labels]

In [ ]:
data_c_per_cls = 555

def get_empty_afp_dict():
    result = dict()
    for a in alphabet_ls:
        result[a] = 0
    
    return result

# Source code credit for this function: https://gist.github.com/shaypal5/94c53d765083101efc0240d776a23823
def print_confusion_matrix_heat_map(confusion_matrix, class_names, figsize = (26,26), fontsize=14, plt_s_path=None):
    """Prints a confusion matrix, as returned by sklearn.metrics.confusion_matrix, as a heatmap.
    
    Arguments
    ---------
    confusion_matrix: numpy.ndarray
        The numpy.ndarray object returned from a call to sklearn.metrics.confusion_matrix. 
        Similarly constructed ndarrays can also be used.
    class_names: list
        An ordered list of class names, in the order they index the given confusion matrix.
    figsize: tuple
        A 2-long tuple, the first value determining the horizontal size of the ouputted figure,
        the second determining the vertical size. Defaults to (10,7).
    fontsize: int
        Font size for axes labels. Defaults to 14.
        
    Returns
    -------
    matplotlib.figure.Figure
        The resulting confusion matrix figure
    """
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('Truth')
    plt.xlabel('Prediction')
    
    if plt_s_path:
        plt.savefig(plt_s_path, dpi=300)

In [ ]:
afp_dict = get_empty_afp_dict()
for pred_cls, gt in zip(pred_res, ground_truths):
    if pred_cls != gt:
        if gt in afp_dict.keys(): # gt should be the key, not pred_cls
            afp_dict[gt] += 1
        else:
            afp_dict[gt] = 0

In [ ]:
afp_df = pd.DataFrame(
    list(afp_dict.items()),
    columns=['alphabet', 'fc'],
)

afp_df['fcr'] = afp_df['fc'] / data_c_per_cls
afp_df['acc'] = 1 - afp_df['fcr']

# Failed Prediciton Count View

## Plotly

### Failed Prediction Count of Each Alphabet

In [ ]:
fig = px.bar(
    afp_df,
    x='alphabet', 
    y='fc',
    
    labels={
        "alphabet": "Alphabet",
        "fc": "Failed Prediction Count",
    },
    title='Failed Prediction Count of Each Alphabet',
)

fig.show()
fig.write_image(fc_chart_s_path)

### Accuarcy of Each Alphabet

In [ ]:
fig = px.bar(
    afp_df,
    x='alphabet', 
    y='acc',
    
    labels={
        "alphabet": "Alphabet",
        "acc": "Accuaracy",
    },
    
    title='Accuarcy of Each Alphabet',
)

fig.show()
fig.write_image(acc_chart_s_path)

### Confusion Matrix Heat Map and F1 Report

In [ ]:
cm = confusion_matrix(ground_truths, pred_res)

In [ ]:
print_confusion_matrix_heat_map(cm, alphabet_ls, plt_s_path=cm_hm_s_path)

In [ ]:
clf_report = classification_report(ground_truths, pred_res)
print(clf_report)

## Other Chart Tools

### matplotlib

In [ ]:
alphabets = afp_df['alphabet'].values
fcs = afp_df['fc'].values
x = np.arange(len(alphabets))
plt.bar(x, fcs)
plt.xticks(x, alphabets)
plt.xlabel('Alphabet')
plt.ylabel('Failed Prediction Count')
plt.title('Failed Prediction Count of Each Alphabet')
plt.savefig(fc_chart_mpl_s_path, dpi=300)
plt.show()

In [ ]:
alphabets = afp_df['alphabet'].values
fcs = afp_df['acc'].values
x = np.arange(len(alphabets))
plt.bar(x, fcs)
plt.xticks(x, alphabets)
plt.xlabel('Alphabet')
plt.ylabel('Accuaracy')
plt.title('Accuarcy of Each Alphabet')
plt.savefig(acc_chart_mpl_s_path, dpi=300)
plt.show()